In [1]:
import os
import xarray as xr
import shutil
from tqdm import tqdm
import logging

# 配置日志
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def process_nc_files(input_dir, output_dir, min_time_dim=24, required_vars=None):
    """
    检查NC文件时间维度和必需变量，并移动不符合要求的文件
    
    参数:
    input_dir: 包含NC文件的目录
    output_dir: 移动不符合要求文件的目标目录
    min_time_dim: 时间维度的最小值要求，默认为24
    required_vars: 需要检查的必需变量列表，默认为None
    """
    # 确保输出目录存在
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        logger.info(f"创建输出目录: {output_dir}")
    
    # 获取所有NC文件
    nc_files = [f for f in os.listdir(input_dir) if f.endswith('.nc')]
    logger.info(f"找到 {len(nc_files)} 个NC文件")
    
    moved_files = 0
    
    # 设置默认的必需变量列表
    if required_vars is None:
        required_vars = ["TAIR", "UWIN", "VWIN", "PRE", "STATION"]
    
    # 处理每个文件
    for filename in tqdm(nc_files, desc="处理文件"):
        file_path = os.path.join(input_dir, filename)
        try:
            # 打开NC文件
            ds = xr.open_dataset(file_path)
            
            should_move = False
            reason = ""
            
            # 检查时间维度
            time_len = len(ds.time)
            if time_len < min_time_dim:
                should_move = True
                reason = f"时间维度: {time_len} < {min_time_dim}"
            
            # 检查必需变量和它们的维度
            missing_vars = []
            wrong_dims_vars = []
            
            for var in required_vars:
                # 检查变量是否存在
                if var not in ds.data_vars:
                    missing_vars.append(var)
                else:
                    # 检查变量维度是否为 (time, lat, lon)
                    var_dims = ds[var].dims
                    expected_dims = ('time', 'lat', 'lon')
                    if var_dims != expected_dims:
                        wrong_dims_vars.append(f"{var}{var_dims}")
            
            if missing_vars or wrong_dims_vars:
                should_move = True
                if missing_vars:
                    if reason:
                        reason += "; "
                    reason += f"缺少变量: {', '.join(missing_vars)}"
                if wrong_dims_vars:
                    if reason:
                        reason += "; "
                    reason += f"变量维度错误: {', '.join(wrong_dims_vars)}"
            
            # 移动不符合要求的文件
            if should_move:
                dest_path = os.path.join(output_dir, filename)
                shutil.move(file_path, dest_path)
                logger.info(f"已移动: {filename} ({reason})")
                moved_files += 1
            else:
                logger.debug(f"保留: {filename} (符合所有要求)")
            
            # 关闭数据集
            ds.close()
            
        except Exception as e:
            logger.error(f"处理 {filename} 时出错: {str(e)}")
            # 可选：把有问题的文件也移动到输出目录
            try:
                dest_path = os.path.join(output_dir, filename)
                shutil.move(file_path, dest_path)
                logger.info(f"已移动出错文件: {filename}")
                moved_files += 1
            except Exception as move_err:
                logger.error(f"移动出错文件 {filename} 失败: {str(move_err)}")
    
    logger.info(f"处理完成。共移动 {moved_files} 个文件到 {output_dir}")

if __name__ == "__main__":
    # 配置参数
    INPUT_DIR = "/mnt/h/DataSet/Merged_padded"  # 修改为你的NC文件目录
    OUTPUT_DIR = "/mnt/h/DataSet/other"         # 修改为你想移动文件的目标目录
    MIN_TIME_DIM = 24                           # 时间维度的最小要求
    
    # 需要检查的变量列表
    REQUIRED_VARS = ["TAIR", "UWIN", "VWIN", "PRE", "STATION"]
    
    # 执行处理
    process_nc_files(INPUT_DIR, OUTPUT_DIR, MIN_TIME_DIM, REQUIRED_VARS)

2025-05-05 20:13:25,310 - INFO - 找到 271 个NC文件
处理文件:   4%|▎         | 10/271 [00:00<00:10, 24.30it/s]2025-05-05 20:13:25,861 - INFO - 已移动: 20220414.nc (时间维度: 17 < 24)
2025-05-05 20:13:25,900 - INFO - 已移动: 20220416.nc (时间维度: 21 < 24)
处理文件:  38%|███▊      | 104/271 [00:03<00:05, 28.48it/s]2025-05-05 20:13:29,203 - INFO - 已移动: 20220717.nc (时间维度: 7 < 24)
2025-05-05 20:13:29,246 - INFO - 已移动: 20220719.nc (时间维度: 23 < 24)
处理文件: 100%|██████████| 271/271 [00:09<00:00, 28.59it/s]
2025-05-05 20:13:34,792 - INFO - 处理完成。共移动 6 个文件到 /mnt/h/DataSet/other


In [3]:
import xarray as xr

# ds = xr.open_dataset("/mnt/h/DataSet/3-DEM/CHN_dem_1km_clipped.nc")
# ds = xr.open_dataset("/mnt/h/DataSet/station_precipitation_data.nc")
ds = xr.open_dataset("/mnt/h/DataSet/Pre/2022-04.nc")
ds

<xarray.Dataset> Size: 1GB
Dimensions:     (valid_time: 720, latitude: 570, longitude: 793)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 6kB 2022-04-01 ... 2022-04-30T23:...
  * latitude    (latitude) float64 5kB 20.02 20.03 20.04 ... 25.69 25.7 25.71
  * longitude   (longitude) float64 6kB 109.5 109.5 109.5 ... 117.4 117.4 117.4
Data variables:
    tp          (valid_time, latitude, longitude) float32 1GB ...
Attributes:
    description:    ERA5降水数据插值到DEM网格
    creation_date:  2025-05-07 11:06:55

In [ ]:
import os
import xarray as xr

def convert_all_tp_to_mm(input_dir, output_dir, months=range(4, 13), year=2022):
    os.makedirs(output_dir, exist_ok=True)
    for month in months:
        input_file = os.path.join(input_dir, f"{year}-{month:02d}.nc")
        output_file = os.path.join(output_dir, f"{year}-{month:02d}.nc")
        if not os.path.exists(input_file):
            print(f"文件不存在: {input_file}")
            continue
        ds = xr.open_dataset(input_file)
        if "tp" not in ds:
            print(f"{input_file} 没有 'tp' 变量，跳过")
            continue
        ds["tp"] = ds["tp"] * 1000
        ds["tp"].attrs["units"] = "mm"
        ds.to_netcdf(output_file)
        print(f"已保存: {output_file}")

# 用法示例
input_dir = "/mnt/h/DataSet/Pre"         # 输入文件夹路径
output_dir = "/mnt/h/DataSet/Pre_mm"     # 输出文件夹路径
convert_all_tp_to_mm(input_dir, output_dir)

已保存: /mnt/h/DataSet/Pre_mm/2022-04.nc
已保存: /mnt/h/DataSet/Pre_mm/2022-05.nc
已保存: /mnt/h/DataSet/Pre_mm/2022-06.nc
已保存: /mnt/h/DataSet/Pre_mm/2022-07.nc
已保存: /mnt/h/DataSet/Pre_mm/2022-08.nc
已保存: /mnt/h/DataSet/Pre_mm/2022-09.nc
已保存: /mnt/h/DataSet/Pre_mm/2022-10.nc
已保存: /mnt/h/DataSet/Pre_mm/2022-11.nc
已保存: /mnt/h/DataSet/Pre_mm/2022-12.nc
